In [1]:
import pandas as pd


In [2]:
# Create a directory of schools
schools_csv_map = [
    {"school_1": "PILTON COMMUNITY COLLEGE",
     "school_2": "GREAT TORRINGTON SCHOOL",
     "school_3": "BRAUNTON SCHOOL AND C.C.",
     "school_4": "HOLSWORTHY COMMUNITY COLLEGE",
     "school_5": "THE ILFRACOMBE ACADEMY"}
]

In [10]:
# Read in data
data = pd.read_csv(
    "data/synthetic/standard/synthetic_standard_survey.standard_school_aggregate_responses.0000000010000.csv"
)

print (data.head())



                    cat                                            cat_lab  \
0  [1, 2, 3, 4, 5, nan]  ['1 - Completely not true', '2', '3', '4', '5 ...   
1  [1, 2, 3, 4, 5, nan]  ['1 - Completely not true', '2', '3', '4', '5 ...   
2  [1, 2, 3, 4, 5, nan]  ['1 - Completely not true', '2', '3', '4', '5 ...   
3  [1, 2, 3, 4, 5, nan]  ['1 - Completely not true', '2', '3', '4', '5 ...   
4  [1, 2, 3, 4, 5, nan]  ['1 - Completely not true', '2', '3', '4', '5 ...   

                      count  \
0  [20, 27, 22, 21, 22, 16]   
1   [26, 29, 21, 23, 26, 3]   
2  [23, 23, 17, 32, 20, 13]   
3   [26, 24, 15, 30, 24, 9]   
4  [31, 22, 22, 19, 21, 13]   

                                          percentage            measure  \
0  [15.625, 21.09375, 17.1875, 16.40625, 17.1875,...  autonomy_pressure   
1  [20.3125, 22.65625, 16.40625, 17.96875, 20.312...   autonomy_express   
2  [17.96875, 17.96875, 13.28125, 25.0, 15.625, 1...    autonomy_decide   
3  [20.3125, 18.75, 11.71875, 23.4375, 18.75

In [ ]:
# Make any changes required (e.g. handling missing data, dropping columns), perform any validation steps

In [ ]:
# Create summary tables

# Section A: summarise RAG ratings for all pupils as a table (only for standard survey)

# Extract counts
counts = data["counts_column_name"]

# School name 

# Chosen group

# Create summary table 
table = summary_table(data, chosen_group, school)


In [ ]:
# Section B graphs: for each topic, generate bar chart and comparison to other schools

In [ ]:
# Section C graphs: who took part

In [ ]:
# Save outputs and repeat for each school and view (i.e. by gender, by school year, FSM, SEN...)